> forward pass vs. decoding phase

- (draft model) token speculation
- (target model) parallel verification
- rejection sampling

-----

- 草稿小模型 (Draft Model) & 目标大模型 (Target Model)
- 推测 (Speculation)：
    - 在当前文本序列 $x$ 的基础上，让草稿模型 M_draft 以自回归的方式快速生成 $K$ 个候选词元（tokens），形成一个草稿序列 $γ = (γ_1, γ_2, ..., γ_K)$。
例如，当前文本是“今天天气”，草稿模型可能快速生成了“真的很好，阳光明媚”。
- 验证 (Verification)：
    - 将原始文本 x 和整个草稿序列 γ 一次性输入给目标模型 M_target。
    - M_target 进行一次前向传播（Forward Pass）。这次计算会并行地输出在每个位置上，它自己“认为”的下一个词的概率分布。也就是说，M_target 会告诉你：
        - 在“今天天气”之后，它想生成的第一个词是什么。
        - 在“今天天气真的”之后，它想生成的第二个词是什么。
        - 在“今天天气真的很好”之后，它想生成的第三个词是什么。
        - ...依此类推。
    - 这里简单展开下（forward pass（已知 input tokens）的情况下是完全可以并行的）
        - 输入: `[T1, T2, T3, T4]`
        - 输出: `[Logits_1, Logits_2, Logits_3, Logits_4]`
        - 这里的关键是 因果注意力遮罩（Causal Attention Mask） 的作用。这个遮罩确保了在计算 Logits_i 时，模型只能“看到” 从 T1 到 T_i 的信息，而不能“偷看”未来的 T_{i+1} 等。
            - Logits_1 是基于 `[T1]` 计算出的，它代表了模型对 T2 的预测。
            - Logits_2 是基于 `[T1, T2]` 计算出的，它代表了模型对 T3 的预测。
            - Logits_3 是基于 `[T1, T2, T3]` 计算出的，它代表了模型对 T4 的预测。
            - Logits_4 是基于 `[T1, T2, T3, T4]` 计算出的，它代表了模型对 T5 的预测。
        - 在常规的自回归生成中，我们只关心最后一个 Logits_4，用它来采样生成 T5。然后把 T5 加入输入，再进行下一次完整的计算。我们把前面计算出的 Logits_1, Logits_2, Logits_3 都“浪费”掉了。
- 比较与接受 (Comparison & Acceptance)：
    - $DP$ (draft prob) vs. $TP$(target prob)
        - if tp >= dp, accepted
    - 从草稿的第一个词 γ_1 开始，逐一进行比较。
        - 比较 γ_1：M_target 在“今天天气”之后想生成的词，和草稿 γ_1（“真的”）是否一致？
            - 如果一致，则接受 γ_1，继续比较下一个。
        - 比较 γ_2：M_target 在“今天天气真的”之后想生成的词，和草稿 γ_2（“很好”）是否一致？
            - 如果一致，则接受 γ_2，继续比较。
        - ...直到出现不匹配：假设在比较 γ_3（“，”）时，M_target 认为在“今天天气真的很好”之后应该生成的是“！”。这时，不匹配发生了。
        - 处理不匹配：我们接受所有匹配的词元（γ_1, γ_2），然后拒绝不匹配的 γ_3 以及其后所有的草稿词元。
- 修正与迭代 (Correction & Iteration)：
模型最终的输出是所有被接受的词元，加上由 M_target 亲自计算出的那个不匹配位置的正确词元。
在上面的例子中，最终输出会增加“真的很好！”。
然后，从这个新的、被验证过的文本序列开始，重复第一步，让草稿模型再次进行推测。

- 上下文 (Prompt): `Why did the chicken` 草稿词元 (Draft Tokens, K=4): cross the farm ?

|  | `cross` | `the` | `farm` | `?` | `To` (额外生成) |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **DP** | 0.7 | 0.9 | 0.8 | 0.8 | - |
| **TP** | 0.9 | 0.9 | 0.7 | 0.9 | 0.8 |
||✔️|✔️|×|×|×||

- 最好的情况，一次 target model forward pass 得到 k+1 个 tokens
- 最坏的情况，一次 target model 的前向也能得到一个 token；